In [2]:
import pandas as pd
import numpy as np
import ipaddress
import dns.resolver
import dns.reversename
import pygeoip
import matplotlib.pyplot as plt 

datafile='test5.parquet'

### IP geolocalization
gi=pygeoip.GeoIP('./GeoIP.dat')
gi2=pygeoip.GeoIP('./GeoIPASNum.dat')
addr='193.136.73.21'
cc=gi.country_code_by_addr(addr)
org=gi2.org_by_addr(addr)
print(cc,org)

### DNS resolution
addr=dns.resolver.resolve("www.ua.pt", 'A')
for a in addr:
    print(a)
    
### Reverse DNS resolution    
name=dns.reversename.from_address("193.136.172.20")
addr=dns.resolver.resolve(name, 'PTR')
for a in addr:
    print(a)

### Read parquet data files
data=pd.read_parquet(datafile)
data=data.head(10000)
#print(data.to_string())

# explore data with head(), info(), describe(), ...

# Number of packets per protocol and ports, and is source CC
Anomalous_nP=data.groupby(['proto','port'])['up_bytes'].count()
Anomalous_nP=data.groupby(['proto','port'])['up_bytes'].count().to_frame(name='nP')
Anomalous_nP=data.groupby(['proto','port'])['up_bytes'].count().reset_index(name='nP')
Anomalous_nP=data.groupby(['proto','port'])['up_bytes'].count().reset_index(name='nP').sort_values(by=['nP'],ascending=False)

# Give me a table of unique source IPs, with the number of packets sent by each one
Anomalous_uniqIP=data.groupby(['src_ip'])['up_bytes'].count().reset_index(name='nP').sort_values(by=['nP'],ascending=False)

# Table with the number of packets exchanged between each pair of IPs
Anomalous_nPbetIPs=data.groupby(['src_ip','dst_ip'])['up_bytes'].count().reset_index(name='nP').sort_values(by=['nP'],ascending=False)

# Calc the min, max, avg, std dev of the number of packets for category of src_ip, dst_ip, proto, port
Anomalous_nPbetIPs=data.groupby(['src_ip','dst_ip','proto','port'])['up_bytes'].count().reset_index(name='nP').sort_values(by=['nP'],ascending=False)

# for each source IP, give me the max number of packets sent to any destination IP
Anomalous_maxP=data.groupby(['src_ip'])['up_bytes'].max().reset_index(name='maxP').sort_values(by=['maxP'],ascending=False)

# for each source IP, give me the minimum number of packets sent to any destination IP, and the destination IP
Anomalous_minP=data.groupby(['src_ip'])['up_bytes'].min().reset_index(name='minP').sort_values(by=['minP'],ascending=False)

# for each source IP, give me the average number of packets sent to any destination IP
Anomalous_avgP=data.groupby(['src_ip'])['up_bytes'].mean().reset_index(name='avgP').sort_values(by=['avgP'],ascending=False)

Anomalous_maxPack=data['up_bytes'].max()

Anomalous_minPack=data['up_bytes'].min()

Anomalous_avgPack=data['up_bytes'].mean()

org=data['dst_ip'].apply(lambda y:gi2.org_by_addr(y)).to_frame(name='org')

sum_packets_by_org=org.groupby(['org'])['org'].count().reset_index(name='count').sort_values(by=['count'],ascending=False)

PT AS1930 Fundacao para a Ciencia e a Tecnologia, I.P.
193.136.173.58
dns.ua.pt.
